# Session 2: Fast machine representation

In this session, we cover the use of fast `Numbers` in Julia.
Particularly:[^1]
- [ ] Demonstrate tradeoff between runtime speed and over- or underflow checks in [number representations in Julia](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/).
- [ ] Analyze the floating point layout or architecture used by your Julia installation.
- [ ] Show how much `@fastmath` macro speeds up computation with a trades off in some level of accuracy. The `sum_diff()` function in the main book reference may be replicated for this purpose.

----
[^1]: Covers Chapter 5 of Segupta, _Julia High Performance, 2nd Ed._ (Packt Publishing, 2019).

In [1]:
using Pkg;
Pkg.activate(".");
Pkg.add([
     "Plots"
    ,"BenchmarkTools"
]);

using Plots, BenchmarkTools;

  Activating project at `~/Documents/GitHub/Phys215-202324-2/02-Performance`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Phys215-202324-2/02-Performance/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202324-2/02-Performance/Manifest.toml`


In [2]:
include("Phys215Tools.jl") #insert pre-typed tool functions, fast and dirty style

floatbits (generic function with 2 methods)

In [3]:
? floatbits

search: floatbits



No documentation found.

`floatbits` is a `Function`.

```
# 2 methods for generic function "floatbits" from Main:
 [1] floatbits(x::Float32)
     @ ~/Documents/GitHub/Phys215-202324-2/02-Performance/Phys215Tools.jl:7
 [2] floatbits(x::Float64)
     @ ~/Documents/GitHub/Phys215-202324-2/02-Performance/Phys215Tools.jl:2
```


## Fast numbers in Julia

> Integers in Julia are stored as system integers.... The `Int` type alias represents the actual integer type used by the system. `Int32` for 32-bit machines; `Int64` for 64-bit machines.[^2]

----
[^2]: Segupta, Julia High Performance, 2nd Ed. (Packt Publishing, 2019).

## Machine bit size and representation

- FOR BASH-like CLI: Use `uname -m` to examine the processor type of your machine.
    - The command `uname -a` provides `a`ll the relevant machine information.
- Default integer representation depends on machine word size.

In [4]:
; uname -vpm

Darwin Kernel Version 23.3.0: Wed Dec 20 21:28:58 PST 2023; root:xnu-10002.81.5~7/RELEASE_X86_64 x86_64 i386


**Note** that the semicolon indicates that the command is a bash command.
You may need to modify that for non-bash CLI.

### System `WORD_SIZE`

- System `WORD_SIZE` becomes the `Int` size (_in bits_) of the Julia installed.
- Check out `? sizeof()` for the output of the command.
- Use `Sys` to indicate namespace or module scoping.

In [5]:
@show Sys.WORD_SIZE;

Sys.WORD_SIZE = 64


### Use `sizeof()` for byte size

- One bit = 1 two-state unit in physical memory
- One byte = 8 bits, 2^8 states in physical memory

In [6]:
? sizeof()

```
sizeof(T::DataType)
sizeof(obj)
```

Size, in bytes, of the canonical binary representation of the given `DataType` `T`, if any. Or the size, in bytes, of object `obj` if it is not a `DataType`.

See also [`Base.summarysize`](@ref).

# Examples

```jldoctest
julia> sizeof(Float32)
4

julia> sizeof(ComplexF64)
16

julia> sizeof(1.0)
8

julia> sizeof(collect(1.0:10.0))
80

julia> struct StructWithPadding
           x::Int64
           flag::Bool
       end

julia> sizeof(StructWithPadding) # not the sum of `sizeof` of fields due to padding
16

julia> sizeof(Int64) + sizeof(Bool) # different from above
9
```

If `DataType` `T` does not have a specific size, an error is thrown.

```jldoctest
julia> sizeof(AbstractArray)
ERROR: Abstract type AbstractArray does not have a definite size.
Stacktrace:
[...]
```

---

```
sizeof(str::AbstractString)
```

Size, in bytes, of the string `str`. Equal to the number of code units in `str` multiplied by the size, in bytes, of one code unit in `str`.

# Examples

```jldoctest
julia> sizeof("")
0

julia> sizeof("∀")
3
```


### `sizeof()` different `Int` type

- `Int` uses the machine default integer size
- Bigger integers may be used up to size 128 bytes (2^1024 physical states total)

In [7]:
@show sizeof(Int); # uses machine's default integer representation
@show sizeof(Int32);
@show sizeof(Int64);
@show sizeof(Int128);

sizeof(Int) = 8
sizeof(Int32) = 4
sizeof(Int64) = 8
sizeof(Int128) = 16


## Machine bit representation of `Int`s

- Similar to base-10 representation for whole numbers
- Applicable only for whole numbers
- Different scheme used for numbers with fractional part: floating-point representation

### Algorithm for finding bit representation

- [Divide by two method](https://en.wikipedia.org/wiki/Binary_number#Decimal_to_binary)

### Sample code

In [8]:
x = 25;

sbit = ""; #initial string, none
while x != 0
    r = rem(x,2); #returns the remainder
    x = div(x,2); #returns the exact division
    sbit = string(r)*sbit; #append to the left
    @show x, r
end

@show sbit;

(x, r) = (12, 1)
(x, r) = (6, 0)
(x, r) = (3, 0)
(x, r) = (1, 1)
(x, r) = (0, 1)
sbit = "11001"


### Checking with `parse()`r function

In [9]:
@show sbit;
@show parse(Int,sbit;base=2);

sbit = "11001"
parse(Int, sbit; base = 2) = 25


### Native function for `bitstring()`
- `bitstring()` function exists within Julia.

In [10]:
? bitstring

search: bitstring SubstitutionString



```
bitstring(n)
```

A string giving the literal bit representation of a primitive type.

See also [`count_ones`](@ref), [`count_zeros`](@ref), [`digits`](@ref).

# Examples

```jldoctest
julia> bitstring(Int32(4))
"00000000000000000000000000000100"

julia> bitstring(2.2)
"0100000000000001100110011001100110011001100110011001100110011010"
```


### Native function for `bitstring()`
- `bitstring()` function exists within Julia.

In [11]:
x = 25;
@show x;
@show bitstring(x);
@show sbit;

x = 25
bitstring(x) = "0000000000000000000000000000000000000000000000000000000000011001"
sbit = "11001"


## Machine representation (IEEE 754 standards)

- Not all numbers perfectly represented in machines
- Binary representation limitations results to under- and overflows
- Floating-point representation in base 2 used for real numbers
- Machine representation covered by [the IEEE Standard for Floating-Point Arithmetic (IEEE 754)](https://en.wikipedia.org/wiki/IEEE_754)
- illustration found in [GeeksForGeeks page (:warning: with paid ads)](https://www.geeksforgeeks.org/ieee-standard-754-floating-point-numbers/).

## Mem size and allocation scheme

Simple `Int` type and `FloatX` type.

In [13]:
println( bitstring(3) );
@show length( bitstring(3) );

0000000000000000000000000000000000000000000000000000000000000011
length(bitstring(3)) = 64


In [14]:
println( bitstring(3.0) );
@show length(bitstring(3.0));

0100000000001000000000000000000000000000000000000000000000000000
length(bitstring(3.0)) = 64


- 💡 Same length; different information.
- 📖 Check out [the IEEE Standard for Floating-Point Arithmetic (IEEE 754)](https://en.wikipedia.org/wiki/IEEE_754) for the bit assignment for `Float64`

## Julia `Numbers` and related functions

- Basic functions provided to allow analysis of `Numbers` representation
- [A range of primitive numeric representations available](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/)
- Also [support for arithmetic that requires arbitrary precision](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Arbitrary-Precision-Arithmetic).

### Checkout `Int` type types

- Default is machine `WORD_LENGTH`.
- Try out different `Int`s.

In [26]:
@show bitstring(Int8(126)); # [-2^7, 2^7-1] (one sign bit, leftmost)
@show bitstring(Int16(126));
@show bitstring(Int32(126));

bitstring(Int8(126)) = "01111110"
bitstring(Int16(126)) = "0000000001111110"
bitstring(Int32(126)) = "00000000000000000000000001111110"


# Fin